In [4]:
import os
import pandas as pd
import numpy as np
import data_wrangle
import json
from importlib import reload
import preprocess


#From PCA
from sklearn.decomposition import PCA
from sklearn import decomposition
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import scipy.signal as ss
import pybaselines.spline as py

import pandas as pd
import numpy as np
from math import ceil


import plotly.io as pio
pio.renderers.default = "browser"

In [8]:
#Reload library to impliment changes
reload(data_wrangle)

#Access dataBase and dataTable
db=data_wrangle.DataTable('/home/dunn/DBM/Brain.db','data')

#Table info
print('Table info without condition set')
db.table_info()

#Set Query Conditions
db.condition='WHERE location IS NOT NULL'

#Table Info
print()
print('WITH CONDITION SET')
db.table_info()
db.columns()


Table info without condition set
DATATABLE INFORMATION
+------------+------+-------------------+-----+-------------------+----+
| Table Name | data | Number of samples | 539 | Number of columns | 16 |
|            |      |       (rows)      |     |                   |    |
+------------+------+-------------------+-----+-------------------+----+
+-------------------------------------------------------------------------------------------------+
|                                        Column Information                                       |
+-------+--------------------+---------------------+---------+---------+------------+-------------+
| colID |        name        | unique_vals_per_col |   type  | notnull | dflt_value | primary_key |
+-------+--------------------+---------------------+---------+---------+------------+-------------+
|   0   |         id         |         539         | INTEGER |    0    |    None    |      1      |
|   1   |    raman_shift     |          12         | 

(['id',
  'raman_shift',
  'original_intensity',
  'frame',
  'filename',
  'location',
  'slice',
  'wavelength',
  'power',
  'objective',
  'time',
  'date',
  'operator_name',
  'raman',
  'general_loc',
  'loc_color'],
 +-------------------------------------------------------------------------------------------------+
 |                                        Column Information                                       |
 +-------+--------------------+---------------------+---------+---------+------------+-------------+
 | colID |        name        | unique_vals_per_col |   type  | notnull | dflt_value | primary_key |
 +-------+--------------------+---------------------+---------+---------+------------+-------------+
 |   0   |         id         |         539         | INTEGER |    0    |    None    |      1      |
 |   1   |    raman_shift     |          12         |   BLOB  |    0    |    None    |      0      |
 |   2   | original_intensity |         446         |   BLOB  |    0 

In [5]:
reload(data_wrangle)
# db=data_wrangle.DataTable('/mnt/c/Users/16162/Desktop/Brain.db','data')
db=data_wrangle.DataTable('/home/dunn/DBM/Brain.db','data')
# db.condition='WHERE strftime(\'%Y\',date)=\'2023\' AND location in (\'substantia nigra\',\'cortex\',\'midbrain\')'
# db.table_info()

db.condition='WHERE location IS NOT NULL'
# db.columns('date','general_loc')[1]



/home/dunn/spinne/DataBase/preprocess.py:46: RuntimeWarning:

divide by zero encountered in true_divide

/home/dunn/spinne/DataBase/preprocess.py:46: RuntimeWarning:

invalid value encountered in true_divide



,0,1,2,3,4,5,6,7,8,9,...,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099
117,-396.027791,-246.644826,-209.348291,-241.682612,-301.192210,-344.590342,-227.813206,-159.457174,-32.876193,-17.367934,...,1105.650766,984.890576,957.588796,976.468422,930.525175,969.854346,1058.503603,1163.617299,1272.554072,1372.672557
118,-162.044481,-149.145070,-169.105442,-208.520687,-253.985895,-291.836416,-318.712074,-254.586025,-188.275582,-124.134853,...,637.384694,638.754446,723.859842,767.462817,706.182451,652.287175,593.305159,583.271789,658.131541,853.828892
119,-168.330410,35.676558,71.128311,-7.994707,-147.712050,-287.251067,-329.453103,-279.209284,-243.670481,-226.983232,...,447.938217,470.117665,529.119405,575.475930,610.182935,650.694990,615.271861,544.827712,442.731257,312.351206
120,-380.119158,-272.167761,-175.671225,-99.588338,-52.877891,-74.661008,-166.495326,-175.362720,-227.755891,-207.366672,...,330.655369,325.855669,408.921615,524.948463,498.784715,427.850303,327.643079,256.667390,255.544464,364.895533
121,-377.651642,-114.094836,-19.140437,-39.893795,-123.460263,-143.717917,-197.075141,-216.184904,-256.172037,-169.477391,...,332.578573,357.015412,308.984545,436.996814,443.082541,480.726594,511.859728,506.707793,445.957676,310.296265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,-212.870419,-252.533927,-259.039495,-246.699224,-229.825211,-263.807477,-242.216417,-318.908142,-277.972527,-214.599017,...,249.992963,286.556383,194.517953,367.392838,282.228668,271.120697,280.465040,293.776862,308.071329,320.363605
536,-349.754744,-235.959399,-158.294364,-112.072009,-92.604703,-128.224298,-53.310982,-134.638888,-110.228898,-137.084580,...,-29.991431,81.830573,106.691621,192.184773,196.245082,209.192883,203.391800,233.285544,327.161982,513.308980
537,-380.552376,-151.568774,-46.908167,-36.983880,-92.209236,-186.620935,-265.923784,-367.805280,-298.290576,-246.816084,...,149.644246,48.368032,-7.656972,49.417703,-2.213153,-23.060378,40.213675,112.507259,172.744600,199.849925
538,-196.819916,-239.126400,-249.563063,-238.204786,-215.126451,-204.389951,-196.280124,-207.722501,-159.201052,-77.472478,...,79.492046,51.234937,-36.061084,65.396183,134.472532,261.146311,357.913184,384.778195,301.791841,69.004620


<h1> PCA

In [3]:
class multivar:
    def __init__(self,intens,rs,color_by,
                 names, #color_dict,
                 hover=None,symbol=None,
                 ncomp=10):
        self.intens=intens #df
        self.rs=rs #series
        
        self.color=list(color_by)
        self.names=list(names)
        self.ncomp=ncomp

        # self.color_dict=color_dict,

        if hover==None:
            self.hover=self.names
        else:
            self.hover=hover
        if symbol==None:
            self.symbol=self.color
        else:
            self.symbol=symbol
        
        self.rs_ax='Raman Shift (cm-1)'

    
    def label_gen(self,prefix,n):
        nums=list(range(1,n+1))
        res=['%s %s'%(prefix, num) for num in nums]
        return res
    
    def symbol_gen(self,len_needed):
        syms=['circle','square','diamond','cross','x','triangle-up','star-triangle-down','triangle-left','triangle-se','pentagon','hexagon2','star','diamond-tall']
        if len_needed <= len(syms):
            return syms[0:len_needed]
        else:
            times_to_mult=ceil((len_needed/len(syms)))
            rep_syms=syms*times_to_mult
            return rep_syms[0:len_needed]

    def pca_run(self):
        pca=decomposition.PCA(n_components=self.ncomp)
        
        #Apply PCA
        X=scale(self.intens)
        X=pca.fit_transform(X)

        #Get components
        pc_labels=self.label_gen('PC',self.ncomp)
        pcs=pca.components_.T*np.sqrt(pca.explained_variance_)
        pcs=pd.DataFrame(pcs,columns=pc_labels, index=self.names)

        #Get explained variance
        explained_list=[round(x*100,2) for x in pca.explained_variance_ratio_]
        explained={pc_labels[i]:explained_list[i] for i in range(self.ncomp)}

        #Get loadings
        loadings=pd.DataFrame(X,columns=pc_labels,index=self.rs)
        return pcs, loadings, explained
    
    def pxfig(self,df,x,y,color,color_dict=None):
        syms=self.symbol_gen(len(df))
        fig=px.scatter(df,x=x,y=y,color=color,
                    #    color_discrete_map=color_dict,
                    #    color_continuous_scale=px.colors.qualitative.T10,
                    #    range_color=(-18,10),
                    #    symbol_sequence=syms, symbol=self.symbol,
                        symbol_sequence=['square','circle','diamond','cross'],
                        symbol=self.color,
                       #symbol_sequence=syms, symbol=self.names, #For color blind friendly figure
                       template='simple_white',hover_name=self.hover)
        
        # fig.update_xaxes(range=[0.8,0.9])
        fig.update_traces(marker_size=10)
        return fig

    def pca_figs(self,x,y,color=None,color_dict=None):
        if color is None:
            color=self.names

        pcs,loadings,explained=self.pca_run()

        #PCA scatter plot
        fig1=self.pxfig(pcs,x,y,color)#,color_dict=color_dict)
        fig1.update_layout(title='%s vs %s'%(x,y),font_size=20)
        fig1.update_xaxes(title_text='%s (%s%s)'%(x,explained[x],'%'),
                          range=[0.8,1])
        fig1.update_yaxes(title_text='%s (%s%s)'%(y,explained[y],'%'),
                          range=[0,0.5])
        
        # save_name='pc_zoom_mix'
        # pio.write_image(fig,'/mnt/c/Users/16162/Desktop/%s.svg'%save_name,
        #         width=1600,
        #         height=800)
        
        #PCA Loadings
        fig2=px.line(loadings,x=loadings.index,y=x,
                    title='%s (%s%s)'%(x,explained[x],'%'), template='simple_white',
                    labels={x:'','index':self.rs_ax})
        #fig.show()
        fig3=px.line(loadings,x=loadings.index,y=y,
                    title='%s (%s%s)'%(y,explained[y],'%'),template='simple_white',
                    labels={y:'','index':self.rs_ax})
        #fig.show()
        return fig1,fig2,fig3
    

    def pca_3d(self, color=None, color_dict=None,sym=None, x='PC 3', y='PC 2', z='PC 1'):
        if color is None:
            color=self.color
        if sym is None:
            pass
        pcs,loadings,explained=self.pca_run()
        
        fig=px.scatter_3d(pcs,x=x,y=y,z=z, color=color,
                        color_discrete_map=color_dict,
                        symbol=self.color, symbol_sequence=['square', 'circle', 'diamond', 'cross'],
                        # color_continuous_scale=px.colors.sequential.Jet, range_color=(0,-20),  #Jet,
                    #   color_continuous_midpoint=3,
                        
                        

                        hover_name=self.hover, 

                        )
        fig.update_layout(height=800)#, template='simple_white')
        fig.update_traces
        # fig.update_layout(scene=dict(xaxis_width=800))
        # fig.update_xaxes(showline=True, linewidth=5, linecolor='black')
        # fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
        # fig.update_zaxes(showline=True, linewidth=2, linecolor='black')
        # fig.write_html('/mnt/c/Users/16162/Desktop/3dpca.html')
        fig.show()

    def pca_fig(self,xpc,ypc):#,color_dict):
        #delete?
        figs=self.pca_figs(xpc,ypc,color=self.color)
        figs[1].show()
        figs[2].show()

    def pca_fig(self,figs):
        fig=make_subplots(rows=2,cols=2,specs=[[{'rowspan':2},{}],[None,{}]],
                          subplot_titles=[
                          figs[0].layout.title.text,figs[1].layout.title.text,figs[2].layout.title.text],
                          vertical_spacing=0.35)
        for trace in figs[0].data:
            fig.add_trace(trace,row=1,col=1)
            fig.update_xaxes(title_text=figs[0].layout.xaxis.title.text,row=1,col=1)
            fig.update_yaxes(title_text=figs[0].layout.yaxis.title.text,row=1,col=1)
        for trace in figs[1].data:
            fig.add_trace(trace,row=1,col=2)
        for trace in figs[2].data:
            fig.add_trace(trace,row=2,col=2)

        fig.update_xaxes(title_text=figs[1].layout.xaxis.title.text,row=1,col=2)
        fig.update_xaxes(title_text=figs[1].layout.xaxis.title.text,row=2,col=2)
        fig.update_xaxes(title_text=figs[0].layout.xaxis.title.text,row=1,col=1)
        fig.update_yaxes(title_text=figs[0].layout.yaxis.title.text,row=1,col=1)

        fig.update_layout(template='simple_white',
                          font_size=20,
                          font_family='ariel')
        return fig

<h1> Data Work

<h2>Get Data From DB

In [4]:
reload(data_wrangle)
db=data_wrangle.DataTable('/home/dunn/DBM/Brain.db','data') #Imported with tobytes
# db=data_wrangle.DataTable('/home/dunn/DBM/Brain.db','data')
# db.condition='WHERE strftime(\'%Y\',date)=\'2023\''
db.condition='WHERE strftime(\'%Y\',date)=\'2023\' AND general_loc in (\'substantia nigra\',\'cortex\',\'midbrain\')'
# db.condition="WHERE date LIKE '2023-%'"# AND location IN ('cortex')"
# db.table_info()
x=db.raman_shifts()
ys=db.intens()
namedic=db.names()


/home/dunn/spinne/DataBase/preprocess.py:46: RuntimeWarning:

divide by zero encountered in true_divide

/home/dunn/spinne/DataBase/preprocess.py:46: RuntimeWarning:

invalid value encountered in true_divide



In [5]:
#Find NaN values
nan_cols=ys.isna().any() #Find columns with NaN values
nan_rows=ys.loc[:,nan_cols].isna().any(axis=1) #Find the rws with NaN values (in columns with NaN values)
nan_rows.index #Get indexes of rows (= sample ID)

#Drop rows with more than 5 NaN values
ys=ys.dropna(thresh=ys.shape[1]-5)

In [6]:
#Map namedic to sample ID
names=ys.index.map(lambda x: namedic[x][2])

In [8]:
ys

,0,1,2,3,4,5,6,7,8,9,...,1330,1331,1332,1333,1334,1335,1336,1337,1338,1339
83,89.317439,53.019025,63.458719,98.848658,137.400976,126.009627,97.007958,9.872172,-37.112001,-6.788705,...,-98.946733,-46.990389,-24.488231,54.304285,89.296207,154.799179,142.499303,120.960750,104.526909,107.541172
84,1345.354054,1006.660499,691.612087,411.991771,179.582504,63.758147,8.393284,-177.879928,-91.844913,60.316634,...,-88.539302,-284.548326,-458.458284,-562.092124,-358.502234,-75.044030,379.853477,655.409185,565.718615,-75.122712
85,229.413103,171.958119,132.535644,106.176026,87.909615,38.455071,61.524560,47.386527,37.872187,15.964269,...,-46.200050,-47.043018,-27.247888,-80.529126,-50.544921,-44.624459,-28.724631,-33.512285,-75.699723,-171.999247
86,41.948622,33.392021,24.203386,15.327165,7.707803,4.594943,3.438802,1.745877,2.494525,2.044055,...,-9.198759,-5.817500,-4.466389,-2.474446,-4.317879,-8.533502,-10.578040,-10.301440,-6.456243,2.205008
87,48.186643,27.642349,15.747408,10.107885,8.329842,4.279083,5.159076,4.692767,2.373666,1.955023,...,1.291885,0.036364,-5.868289,-7.400451,-7.581788,-6.793276,-5.195110,-6.339259,-12.316654,-25.218228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,-515.576447,-342.581126,-331.550947,-413.904161,-521.059018,-486.622080,-423.633532,-321.166026,-92.006500,194.694474,...,-309.525151,-313.036151,-369.460910,-421.056176,-477.238872,-604.824188,-740.941165,-783.046409,-664.434185,-318.398760
323,-364.414974,-101.169303,-76.359019,-209.932769,-421.839203,-621.494501,-640.555825,-604.256096,-421.226502,-113.293035,...,-73.032746,-233.506543,-393.195809,-414.855237,-700.321768,-676.878235,-650.240366,-642.655639,-698.968933,-864.025128
324,-247.652622,-240.518936,-319.458934,-436.431313,-543.394767,-485.307994,-495.877884,-331.878430,-232.356350,-155.094295,...,-179.192423,-296.247555,-296.795075,-374.693574,-490.663116,-519.618566,-593.440162,-692.200778,-825.765498,-1003.999408
325,-354.299742,-253.242650,-275.261387,-368.456079,-480.926850,-519.027071,-454.684428,-341.512754,-268.827182,-108.414705,...,-165.596437,-241.458246,-323.648610,-523.943820,-601.964324,-635.360870,-705.944382,-738.044540,-712.672846,-610.840799


<h1> Analysis

In [7]:
# pio.renderers.default = "browser"
var=multivar(ys.T,x,names,names)
# figs=var.pca_fig('PC 1','PC 2',ys.index)
# figs=var.pca_figs('PC 2', 'PC 3')
# var.pca_fig(figs)
# var.pca_fig
var.pca_3d(color=names)